In this jupyter notebook, we will be analyzing plate reader data from experiments used to characterize leaky expression from the ColE1 plasmid encoding T7 RNAP-driven GFP in the absence of T7 RNAP, and generate the plot as shown in Supplementary Figure 29.

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize
import murraylab_tools.biotek as btek
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler
import numpy as np

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('legend', frameon=False)
mpl.rc('figure', figsize=(8.5,15))
%matplotlib qt

First load data/metadata

In [2]:
# load data
endpoint_data = pd.read_csv('./20211005_leak_experiment_endpoint.csv')
endpoint_metadata = pd.read_csv('./20211005_metadata.csv')
columns = list(endpoint_metadata.columns)
columns += ['channel','gain','measurement']
cols = np.arange(1,13)
rows = ['A','B','C','D','E','F','G']
reads = endpoint_data['read'].unique()
data = []
df = endpoint_data
for row in rows:
    for col in cols:
        for read in reads:
            value = df.loc[(df.row==row)&(df.read==read),str(int(col))].values[0]
            channel = read.split('_')[0]
            gain = read.split('_')[1]
            data_add = list(endpoint_metadata.loc[endpoint_metadata.well==row+str(int(col)),:].values.flatten())
            data_add+=[channel,gain,value]
            data.append(data_add)
df_master = pd.DataFrame(columns=columns,data=data)
df_master['gain'] = df_master['gain'].astype('int')
df_master.drop(index=df_master.loc[df_master.strain=='empty'].index,inplace=True)   
df_od = df_master.loc[df_master.channel=='OD600',:]
df_od['measurement'] = df_od['measurement'].astype('float64') 
df_gfp = df_master.loc[(df_master.channel=='sfGFP')&(df_master.gain==2),:]
df_gfp['measurement'] = df_gfp['measurement'].astype('float64')
od_bg = df_od.loc[df_od.strain=='media','measurement'].mean()
df_od['measurement'] -= od_bg
df_od.drop(index=df_od.loc[df_od.strain=='media'].index,inplace=True)     
df_gfp.drop(index=df_gfp.loc[df_gfp.strain=='media'].index,inplace=True)    
df_gfp['GFP/OD'] = df_gfp.measurement.values/df_od.measurement.values #OD normalized GFP fluorescence

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [3]:
df_od.to_csv('figureS29_OD600.csv')
df_gfp.to_csv('figureS29_GFP_normGFP.csv')

Below we generate the plot as shown in Supplementary Figure 29. Note that the colors for the negative control plot have been manually changed in Affinity Designer. Though we include an uninsulated construct on the AmpR backbone here, the main comparison is between uninsulated and insulated versions on the KanR backbone. From other experiments we show that AmpR plasmids may be more susceptible to loss/copy number fluctuations due to communal antibiotic degradation. We also have a horizontal plot to more easily display the plasmid diagrams.

In [3]:
sns.set_palette('colorblind',4)
fig, ax = plt.subplots(figsize=(6,5))  
df_plot = df_gfp.loc[df_gfp.iptg==0,:]
strains = np.array(['1x diff 2x bxb1','split pir 2x bxb1'])

sns.stripplot(y="plasmid", x='GFP/OD', hue="strain", data=df_plot,dodge=True,orient='h')
sns.pointplot(y="plasmid", x='GFP/OD', hue="strain", data=df_plot, dodge=0.8 -0.8/2,
              join=False, palette="dark",
              markers="d", scale=1, ci=None,orient='h')
ax.set_xscale('log')
plt.savefig('20211005_leak_horizontal_20220102.pdf')